In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

## load the libraries 
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D, Flatten, Dropout, concatenate,GlobalAveragePooling2D, AveragePooling2D, Activation, Add, LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.normalization import BatchNormalization
from keras.activations import relu
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.models import Model
from keras.initializers import glorot_uniform, Constant
from keras import optimizers
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Optimizer
from keras.utils.generic_utils import get_custom_objects
from keras_tqdm import TQDMNotebookCallback

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from hyperopt import Trials, STATUS_OK, tpe

import gc;

from hyperas import optim
from hyperas.distributions import choice, uniform, lognormal, normal, loguniform

from hyperas.distributions import uniform

C:\Users\Infernal\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    nb_classes = 10
    # the data, shuffled and split between train and test sets
    train = pd.read_csv("data/fashion/fashion-mnist_train.csv")
    test = pd.read_csv("data/fashion/fashion-mnist_test.csv")

    train_x = train[list(train.columns)[1:]].values
    train_y = train['label'].values
    test_x = test[list(test.columns)[1:]].values
    test_y = test['label'].values

    train_x=train_x.reshape(60000,28,28,1)
    test_x=test_x.reshape(10000,28,28,1)
    
    train_y = to_categorical(train_y, num_classes = 10)
    test_y = to_categorical(test_y, num_classes = 10)
    
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=int(train_x.shape[0]/5))


    train_x = train_x / 255
    val_x = val_x / 255
    test_x = test_x / 255

    return train_x, train_y, val_x, val_y, test_x, test_y

In [3]:
def main_block(x, filters, n, strides, activation, initializer, dropout):
    # Normal part
    x_res = Conv2D(filters, (3,3), strides=strides, padding="same", kernel_initializer=initializer, kernel_regularizer=l2(5e-4))(x)
    x_res = BatchNormalization()(x_res)
    x_res = activation()(x_res)
    x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
    # Alternative branch
    x = Conv2D(filters, (1,1), strides=strides)(x)
    # Merge Branches
    x = Add()([x_res, x])

    for i in range(n-1):
        # Residual conection
        x_res = BatchNormalization()(x)
        x_res = activation()(x_res)
        x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
        # Apply dropout if given
        if dropout: x_res = Dropout(rate=dropout)(x)
        # Second part
        x_res = BatchNormalization()(x_res)
        x_res = activation()(x_res)
        x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
        # Merge branches
        x = Add()([x, x_res])

    # Inter block part
    x = BatchNormalization()(x)
    x = activation()(x)
    return x

def build_model(input_dims, output_dim, n, k, activation, initializer, dropout=None):
    assert (n-4)%6 == 0
    assert k%2 == 0
    n = (n-4)//6 
    
    #1)Input
    inputs = Input(shape=(input_dims))

    #2)Initial part
    x = Conv2D(16, (3,3), padding="same", kernel_initializer=initializer)(inputs)
    x = BatchNormalization()(x)
    x = activation()(x)

    #3)wide blocks (can be made into for loop)
    x = main_block(x, 16*k, n, (1,1), activation, initializer, dropout)
    x = main_block(x, 32*k, n, (2,2), activation, initializer, dropout)
    x = main_block(x, 64*k, n, (2,2), activation, initializer, dropout)

    #4)Final part
    x = AveragePooling2D((7,7))(x)
    x = Flatten()(x)
    outputs = Dense(output_dim, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
def model(train_x, train_y, val_x, val_y, test_x, test_y):
    architecture={{choice([(22,8), (28,10), (40,4)])}}
    activation={{choice([ReLU, LeakyReLU, PReLU, ELU, ThresholdedReLU])}}
    initializer={{choice(['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])}}
    dropout={{uniform(0, 0.5)}}
    
    learning_rate=10**{{uniform(-3, 0)}}
    opt={{choice([optimizers.Adam, optimizers.RMSprop, optimizers.Adadelta, optimizers.Nadam, optimizers.Adamax])}}
    if architecture[0]==28:
        batch={{choice([32, 64])}}
    else:
        batch={{choice([32, 64, 128])}}
    
    
    print("""
          Hyperparameters Used \n\n\n
          -----------------------------
          model: ({},{})
          activation: {}
          initializer: {}
          dropout: {}
          learning rate: {}
          optimizer: {}
          batch size: {}
          -----------------------------
          """
          .format(architecture[0], architecture[1], activation, initializer, dropout, learning_rate, opt, batch))
    
    model = build_model((28,28,1), 10, architecture[0], architecture[1], activation, initializer, dropout)
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=0),
                 ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5,min_lr=0.00001),
                 TQDMNotebookCallback()]

    model.compile(optimizer=opt(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_x, train_y, validation_data = (val_x, val_y), callbacks=callbacks, batch_size=batch, epochs=1, verbose=0)

    score, acc = model.evaluate(test_x, test_y, verbose=0)
    
    y_val_pred = model.predict(val_x)
    print("Validation accuracy: ", accuracy_score(np.argmax(val_y, axis=1), np.argmax(y_val_pred, axis=1)))
    
    y_test_pred = model.predict(test_x)
    print("Test accuracy: ", accuracy_score(np.argmax(test_y, axis=1), np.argmax(y_test_pred, axis=1)), "\n\n")
    
    K.clear_session()
    gc.collect()
    del model
    
    return {'loss': acc, 'status': STATUS_OK}


In [5]:
if __name__ == '__main__':
    functions=[main_block, build_model]

    best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          functions=functions,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          trials=Trials(),
                                          notebook_name="WRN Hyperas")

    train_x, train_y, val_x, val_y, test_x, test_y = data()
    print("Best run: ", best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import keras.backend as K
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D, Flatten, Dropout, concatenate, GlobalAveragePooling2D, AveragePooling2D, Activation, Add, LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
except:
    pass

try:
    from keras.layers.normalization import BatchNormalization
except:
    pass

try:
    from keras.activations import relu
except:
    pass

try:
    from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.initializ

          
  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From C:\Users\Infernal\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


Validation accuracy:                                
0.3365                                              
Test accuracy:                                      
0.3352                                              
                                                                  
          Hyperparameters Used 



          -----------------------------
          model: (40,4)
          activation: <class 'keras.layers.advanced_activations.LeakyReLU'>
          initializer: glorot_uniform
          dropout: 0.41833334235579095
          learning rate: 0.5476313990663524
          optimizer: <class 'keras.optimizers.Adamax'>
          batch size: 128
          -----------------------------
          
  5%|▌         | 1/20 [02:37<49:51, 157.47s/it, best loss: 0.3352]

Validation accuracy:                                              
0.09825                                                           
Test accuracy:                                                    
0.1                                                               
                                                                  
          Hyperparameters Used 



          -----------------------------
          model: (28,10)
          activation: <class 'keras.layers.advanced_activations.ThresholdedReLU'>
          initializer: glorot_uniform
          dropout: 0.23634271098298437
          learning rate: 0.152291090611222
          optimizer: <class 'keras.optimizers.Adadelta'>
          batch size: 32
          -----------------------------
          
 10%|█         | 2/20 [05:00<45:57, 153.19s/it, best loss: 0.1]

Validation accuracy:                                           
0.809                                                          
Test accuracy:                                                 
0.8085                                                         
                                                                 
          Hyperparameters Used 



          -----------------------------
          model: (28,10)
          activation: <class 'keras.layers.advanced_activations.LeakyReLU'>
          initializer: glorot_uniform
          dropout: 0.4956006935248156
          learning rate: 0.19672068575140908
          optimizer: <class 'keras.optimizers.RMSprop'>
          batch size: 32
          -----------------------------
          
 15%|█▌        | 3/20 [12:09<1:06:50, 235.91s/it, best loss: 0.1]

...

Validation accuracy:                                              
0.5960833333333333                                                
Test accuracy:                                                    
0.6041                                                            
                                                                  
          Hyperparameters Used 



          -----------------------------
          model: (28,10)
          activation: <class 'keras.layers.advanced_activations.ThresholdedReLU'>
          initializer: he_normal
          dropout: 0.005545311659215679
          learning rate: 0.47370813020083036
          optimizer: <class 'keras.optimizers.RMSprop'>
          batch size: 64
          -----------------------------
          
 80%|████████  | 16/20 [1:08:48<19:00, 285.14s/it, best loss: 0.1]

Validation accuracy:                                              
0.09975                                                           
Test accuracy:                                                    
0.1                                                               
                                                                  
          Hyperparameters Used 



          -----------------------------
          model: (40,4)
          activation: <class 'keras.layers.advanced_activations.ThresholdedReLU'>
          initializer: he_uniform
          dropout: 0.27710396518920993
          learning rate: 0.2844329399628118
          optimizer: <class 'keras.optimizers.Nadam'>
          batch size: 64
          -----------------------------
          
 85%|████████▌ | 17/20 [1:14:33<15:09, 303.03s/it, best loss: 0.1]

Validation accuracy:                                              
0.10266666666666667                                               
Test accuracy:                                                    
0.1                                                               
                                                                  
          Hyperparameters Used 



          -----------------------------
          model: (40,4)
          activation: <class 'keras.layers.advanced_activations.ELU'>
          initializer: glorot_normal
          dropout: 0.1820122957634046
          learning rate: 0.004830278996280274
          optimizer: <class 'keras.optimizers.Adamax'>
          batch size: 64
          -----------------------------
          
 90%|█████████ | 18/20 [1:17:12<08:40, 260.02s/it, best loss: 0.1]

Validation accuracy:                                              
0.7711666666666667                                                
Test accuracy:                                                    
0.7755                                                            
                                                                  
          Hyperparameters Used 



          -----------------------------
          model: (22,8)
          activation: <class 'keras.layers.advanced_activations.PReLU'>
          initializer: glorot_uniform
          dropout: 0.27075464657505727
          learning rate: 0.031570602270260095
          optimizer: <class 'keras.optimizers.RMSprop'>
          batch size: 64
          -----------------------------
          
 95%|█████████▌| 19/20 [1:19:39<03:45, 225.91s/it, best loss: 0.1]

Validation accuracy:                                              
0.792                                                             
Test accuracy:                                                    
0.7969                                                            
100%|██████████| 20/20 [1:23:28<00:00, 226.88s/it, best loss: 0.1]
Best run:  {'activation': 1, 'architecture': 2, 'batch': 0, 'batch_1': 2, 'dropout': 0.41833334235579095, 'dropout_1': -0.2615116590582889, 'initializer': 2, 'opt': 4}
